![alt text](htop.png "Title")

When training a model on a large dataset that does not fit into memory, you can write a custom generator that can pass the data to the model in batches. Once you have the generator, fitting the model is as simple as calling ```fit_generator()``` instead of ```fit()``` on the Keras model. You can include data preprocessing or data augmentation in the generator.

As of Keras 2.0.6, a [Sequence](https://keras.io/utils/#sequence) object is available that allows for safe multiprocessing:

> Sequence are a safer way to do multiprocessing. This structure guarantees that the network will only train once on each sample per epoch which is not the case with generators.

Allowing for multiprocessing can lead to large speedups in training and can help to avoid bottlenecking a GPU. This notebook contains an minnimal working example on how to use a ```Sequence``` object to fit a Keras model.

In [1]:
import os

import tensorflow as tf
from tensorflow import keras
from keras.utils import Sequence

import pandas as pd
import numpy as np

Using TensorFlow backend.


We will try to classify whether an MNIST image contains a 0 or an 8. The raw images should sit in the folder ```./im```. You can download the images in .jpg format from https://www.kaggle.com/scolianni/mnistasjpg#trainingSet.tar.gz. I converted the labels into a .csv file which you can find in this repository. These files provide a mapping from image names to labels (a 0 is labelled as 0, an 8 as 1 in the dataframes):

In [2]:
train = pd.read_csv('./TRAIN.csv')
train.head(10)

,image_name,label
0,img_31624.jpg,1
1,img_4624.jpg,0
2,img_12582.jpg,1
3,img_38294.jpg,0
4,img_16362.jpg,1
5,img_635.jpg,0
6,img_37753.jpg,0
7,img_30343.jpg,1
8,img_26516.jpg,1
9,img_23739.jpg,1


This is the example code from the Keras docs:

```
class CIFAR10Sequence(Sequence):

    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

        return np.array([
            resize(imread(file_name), (200, 200))
               for file_name in batch_x]), np.array(batch_y)
```

To use this code, I will add the following methods:
    * `on_epoch_end` to shuffle the indices after each epoch when in training mode
    * `get_batch_labels` and `get_batch_features` will implement the necessary logic which is different for the labels (which are held in memory) and the images (which are read from disk).

In [3]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import math
import random

datapath = './im/'

def load_image(im):
    return img_to_array(load_img(im, grayscale=True)) / 255.

class DataSequence(Sequence):
    """
    Keras Sequence object to train a model on larger-than-memory data.
    """
    def __init__(self, df, data_path, batch_size, mode='train'):
        self.df = df
        self.bsz = batch_size
        self.mode = mode

        # Take labels and a list of image locations in memory
        self.labels = self.df['label'].values
        self.im_list = self.df['image_name'].apply(lambda x: os.path.join(data_path, x)).tolist()

    def __len__(self):
        return int(math.ceil(len(self.df) / float(self.bsz)))

    def on_epoch_end(self):
        # Shuffles indexes after each epoch if in training mode
        self.indexes = range(len(self.im_list))
        if self.mode == 'train':
            self.indexes = random.sample(self.indexes, k=len(self.indexes))

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return self.labels[idx * self.bsz: (idx + 1) * self.bsz]

    def get_batch_features(self, idx):
        # Fetch a batch of inputs
        return np.array([load_image(im) for im in self.im_list[idx * self.bsz: (1 + idx) * self.bsz]])

    def __getitem__(self, idx):
        batch_x = self.get_batch_features(idx)
        batch_y = self.get_batch_labels(idx)
        return batch_x, batch_y

Let's construct a simple Keras model:

In [4]:
from keras.models import Model
from keras.layers import Input, Conv2D, Dense, Dropout, MaxPool2D, Flatten

im_size = 28

x = Input(shape=(im_size,im_size, 1))
conv_1 = MaxPool2D()(Conv2D(32, (3,3), activation='relu')(x))
conv_2 = MaxPool2D()(Conv2D(32, (3,3), activation='relu')(conv_1))
conv_3 = MaxPool2D()(Conv2D(32, (3,3), activation='relu')(conv_2))
flat = Flatten()(conv_3)
dense_1 = Dropout(0.2)(Dense(32, activation='relu')(flat))
output = Dense(1, activation='sigmoid')(dense_1)

model = Model(inputs=x, outputs=output)
model.compile(optimizer='sgd', loss='binary_crossentropy')

Finally, train the model with the Sequence object and allow for multiprocessing:

In [5]:
seq = DataSequence(train, './im',  batch_size=20)
model.fit_generator(seq, epochs=5, verbose=1, use_multiprocessing=False, workers=1)

Epoch 1/5
410/410 [==============================] - 6s 14ms/step - loss: 0.4353
Epoch 2/5
410/410 [==============================] - 5s 13ms/step - loss: 0.0830
Epoch 3/5
410/410 [==============================] - 6s 13ms/step - loss: 0.0511
Epoch 4/5
410/410 [==============================] - 6s 14ms/step - loss: 0.0372
Epoch 5/5
410/410 [==============================] - 6s 15ms/step - loss: 0.0278
